# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 7:48AM,260910,20,REL1-5079-B-B-B,Reliable 1 Laboratories LLC,Released
1,Apr 19 2023 7:48AM,260910,20,REL1-5420-B-B,Reliable 1 Laboratories LLC,Released
2,Apr 19 2023 7:48AM,260910,20,REL1-5826,Reliable 1 Laboratories LLC,Released
3,Apr 19 2023 7:48AM,260910,20,REL1-5827,Reliable 1 Laboratories LLC,Released
4,Apr 19 2023 7:48AM,260910,20,REL1-5828,Reliable 1 Laboratories LLC,Released
5,Apr 19 2023 7:48AM,260910,20,REL1-5829,Reliable 1 Laboratories LLC,Released
6,Apr 19 2023 7:48AM,260910,20,REL1-5830,Reliable 1 Laboratories LLC,Released
7,Apr 19 2023 7:48AM,260910,20,REL1-5831,Reliable 1 Laboratories LLC,Released
8,Apr 19 2023 7:48AM,260910,20,REL1-5832,Reliable 1 Laboratories LLC,Released
9,Apr 19 2023 7:48AM,260910,20,REL1-5833,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,261343,Released,1
8,261351,Released,1
9,261368,Released,1
10,261370,Released,1
11,261376,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261343,NaN,NaN,1.0
261351,NaN,NaN,1.0
261368,NaN,NaN,1.0
261370,NaN,NaN,1.0
261376,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260869,0.0,0.0,33.0
260910,0.0,0.0,25.0
261273,11.0,0.0,1.0
261300,0.0,0.0,1.0
261331,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260869,0,0,33
260910,0,0,25
261273,11,0,1
261300,0,0,1
261331,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260869,0,0,33
1,260910,0,0,25
2,261273,11,0,1
3,261300,0,0,1
4,261331,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260869,,,33
1,260910,,,25
2,261273,11,,1
3,261300,,,1
4,261331,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 7:48AM,260910,20,Reliable 1 Laboratories LLC
25,Apr 19 2023 7:46AM,261376,22,"NBTY Global, Inc."
26,Apr 19 2023 7:23AM,260869,20,Reliable 1 Laboratories LLC
59,Apr 18 2023 4:32PM,261370,22,"NBTY Global, Inc."
60,Apr 18 2023 4:30PM,261368,12,"SD Head USA, LLC"
61,Apr 18 2023 3:00PM,261351,10,ISDIN Corporation
62,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC"
63,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc."
90,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line"
91,Apr 18 2023 10:01AM,261300,20,"Andelyn Biosciences, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 19 2023 7:48AM,260910,20,Reliable 1 Laboratories LLC,,,25
1,Apr 19 2023 7:46AM,261376,22,"NBTY Global, Inc.",,1,
2,Apr 19 2023 7:23AM,260869,20,Reliable 1 Laboratories LLC,,,33
3,Apr 18 2023 4:32PM,261370,22,"NBTY Global, Inc.",,,1
4,Apr 18 2023 4:30PM,261368,12,"SD Head USA, LLC",,,1
5,Apr 18 2023 3:00PM,261351,10,ISDIN Corporation,,,1
6,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",,,1
7,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",,,27
8,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",,1,
9,Apr 18 2023 10:01AM,261300,20,"Andelyn Biosciences, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 7:48AM,260910,20,Reliable 1 Laboratories LLC,25,,
1,Apr 19 2023 7:46AM,261376,22,"NBTY Global, Inc.",,1,
2,Apr 19 2023 7:23AM,260869,20,Reliable 1 Laboratories LLC,33,,
3,Apr 18 2023 4:32PM,261370,22,"NBTY Global, Inc.",1,,
4,Apr 18 2023 4:30PM,261368,12,"SD Head USA, LLC",1,,
5,Apr 18 2023 3:00PM,261351,10,ISDIN Corporation,1,,
6,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",1,,
7,Apr 18 2023 12:13PM,261334,19,"GCH Granules USA, Inc.",27,,
8,Apr 18 2023 12:05PM,261331,10,"Uniderm USA, Inc - Collagenil Line",,1,
9,Apr 18 2023 10:01AM,261300,20,"Andelyn Biosciences, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 7:48AM,260910,20,Reliable 1 Laboratories LLC,25,,
1,Apr 19 2023 7:46AM,261376,22,"NBTY Global, Inc.",,1,
2,Apr 19 2023 7:23AM,260869,20,Reliable 1 Laboratories LLC,33,,
3,Apr 18 2023 4:32PM,261370,22,"NBTY Global, Inc.",1,,
4,Apr 18 2023 4:30PM,261368,12,"SD Head USA, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 7:48AM,260910,20,Reliable 1 Laboratories LLC,25.0,NaN,NaN
1,Apr 19 2023 7:46AM,261376,22,"NBTY Global, Inc.",NaN,1.0,NaN
2,Apr 19 2023 7:23AM,260869,20,Reliable 1 Laboratories LLC,33.0,NaN,NaN
3,Apr 18 2023 4:32PM,261370,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Apr 18 2023 4:30PM,261368,12,"SD Head USA, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 19 2023 7:48AM,260910,20,Reliable 1 Laboratories LLC,25.0,0.0,0.0
1,Apr 19 2023 7:46AM,261376,22,"NBTY Global, Inc.",0.0,1.0,0.0
2,Apr 19 2023 7:23AM,260869,20,Reliable 1 Laboratories LLC,33.0,0.0,0.0
3,Apr 18 2023 4:32PM,261370,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Apr 18 2023 4:30PM,261368,12,"SD Head USA, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,522682,1.0,1.0,0.0
12,261368,1.0,0.0,0.0
19,783950,29.0,0.0,11.0
20,783079,59.0,0.0,0.0
22,522746,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,522682,1.0,1.0,0.0
1,12,261368,1.0,0.0,0.0
2,19,783950,29.0,0.0,11.0
3,20,783079,59.0,0.0,0.0
4,22,522746,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1.0,1.0,0.0
1,12,1.0,0.0,0.0
2,19,29.0,0.0,11.0
3,20,59.0,0.0,0.0
4,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1.0
1,12,Released,1.0
2,19,Released,29.0
3,20,Released,59.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Completed,0.0,0.0,11.0,0.0,0.0
Executing,1.0,0.0,0.0,0.0,1.0
Released,1.0,1.0,29.0,59.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Completed,0.0,0.0,11.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,1.0
2,Released,1.0,1.0,29.0,59.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Completed,0.0,0.0,11.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,1.0
2,Released,1.0,1.0,29.0,59.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()